In [1]:
from __future__ import division
import pandas as pd
import os

from radd.bk import react, fitfx, utils

# Load Reformatted ssBayes Data

In [3]:
os.chdir("/Users/kyle/Dropbox/CoAx/SSBayes/raw_beh")

#load early gaussian group data 
ssb_e=pd.read_csv("ssb_early.csv")

#load late gaussian group data 
ssb_l=pd.read_csv("ssb_late.csv")

#load uniform group data 
ssb_u=pd.read_csv("ssb_uniform.csv")

inits={'a': 0.4315506305896373,
 'ssv': 0.9698666206553722,
 't': 0.30656375144494813,
 'v': .9,
 'z': 0.16611261860323445}

In [14]:
#load early gaussian group data 
ssb_e=pd.read_csv("ssb_early.csv")
df=ssb_e.copy()

idx_list=df.idx.unique()


dfsx1=df[df['idx'].isin(idx_list[:5])]
dfsx2=df[df['idx'].isin(idx_list[5:10])]
dfsx3=df[df['idx'].isin(idx_list[10:15])]
dfsx4=df[df['idx'].isin(idx_list[15:20])]
dfsx5=df[df['idx'].isin(idx_list[20:])]                

In [99]:
react=reload(react)

flatdf=ssb_e.copy()
flatdf['idx']=np.ones_like(flatdf['idx'])*1000

pred = react.fit_reactive_sx(flatdf, inits, ntrials=100, all_params=1)

# Reformat ssBayes Data for RaDD

In [34]:
#All datasets were formatted as follows:
os.chdir("/Users/kyle/Dropbox/CoAx/SSBayes/raw_beh")

#read in original
early=pd.read_csv("SSBayes_EarlyGaussianGroup.dat", delim_whitespace=True)
late=pd.read_csv("SSBayes_LateGaussianGroup.dat", delim_whitespace=True)
uniform=pd.read_csv("SSBayes_UniformGroup.dat", delim_whitespace=True)

group_dfs={"early":early, "late":late, "uniform":uniform}

In [49]:
for group in group_dfs.keys():
    
    orig=group_dfs[group]

    df=orig.rename(columns={'RT':'rt_msec', 'RT_sec':'rt', 'Keypress':'response'}, axis=1)
    # delete all stop-signal trials that were drawn from the context pdf
    # (Leaves only the probe stop trials and go trials (no stop signal appeared))
    df=df[df['StopSignal'].isin([0.0, 0.4, 0.5, 0.6, 0.7, 0.8])]

    df.rename(columns={'SubjID':'idx','StopTrial':'trial_type', 
                       'StopSignal':'ssd_perc', 'rt_msec':'rt'}, inplace=True)
    df['ssd']=df['ssd_perc']*.500
    
    dfgo=df[df.loc[:,'response']==0]
    dfstop=df[df.loc[:,'response']==1]
    dfgo.loc[:,'response']=np.ones(len(dfgo))
    dfstop.loc[:,'response']=np.zeros(len(dfstop))
    
    df.replace({'trial_type':{0:'go', 1:'stop'}}, inplace=True)
    
    df['choice']=df['response'].copy()
    df.ix[(df['response']==0), 'choice']='go'
    df.ix[(df['response']==1), 'choice']='stop'
    df.replace({'response':{0:5, 1:6}}, inplace=True)
    df.replace({'response':{5:1, 6:0}}, inplace=True)
    #edf['choice']=edf['response'].copy()
    df['acc']=np.zeros(len(df))

    df.ix[(df['choice']=='stop')&(df['trial_type']=='stop'), 'acc']=1
    df.ix[(df['choice']=='go')&(df['trial_type']=='go'), 'acc']=1
    #df.to_csv("ssb_"+group+".csv",index=False)